## Uploading an ONNX model to deepmirror

Before running this notebook, please ensure you:

1. Are logged in by running `dm login EMAIL` in the terminal
2. Have a Token registered and saved on the file


In [ ]:
!pip install deepmirror
# !dm login <YOUREMAIL>

In [ ]:
import tempfile

import onnx
import pandas as pd
from onnx import TensorProto, helper

import deepmirror.api as api

### Build an exmaple onnx model

In [ ]:
def create_onnx_model(
    ir_version: int = 10, opset_version: int = 22, input_dim: int = 1024
) -> onnx.ModelProto:
    X = helper.make_tensor_value_info(
        "input", TensorProto.FLOAT, [1, input_dim]
    )
    Y = helper.make_tensor_value_info("output", TensorProto.FLOAT, [1])
    node = helper.make_node("Identity", inputs=["input"], outputs=["output"])
    graph = helper.make_graph([node], "test", [X], [Y])
    model = helper.make_model(
        graph,
        ir_version=ir_version,
        opset_imports=[helper.make_operatorsetid("", opset_version)],
    )
    return model

In [ ]:
model = create_onnx_model()

In [ ]:
with tempfile.NamedTemporaryFile(suffix=".onnx") as temp_file:
    onnx.save(model, temp_file.name)
    model = api.upload_onnx_model(temp_file.name)

In [ ]:
model

In [ ]:
data = [
    "C[C@H](C(=O)O)[C@@H](NC(=O)c1ncnc2[nH]ccc12)c1ccc(Br)cc1",
    "CC(C)[C@H](Nc1ncnc(N)c1Cl)c1ccc2c(c1)S(=O)(=O)CCC2",
    "CNC(=O)c1cc2c(N[C@H](c3ccc4c(c3)OCCO4)C(C)C)ncnc2[nH]1",
    "CC(C)[C@@H](Nc1ncnc2[nH]c(-c3cnn(C)c3C#N)cc12)c1ccc2c(c1)OCCO2",
    "CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H]2Cc2nn[nH]n2)c2cc(Cl)ccc2C1=O",
    "O=C(Nc1cccc(B(O)O)c1)[C@H]1CCN(Cc2ccccc2)C1",
    "O=C(O)Cc1ccccc1Nc1c(Cl)cccc1Cl",
    "O=C(Nc1ccc2c(c1)CNC2)c1cc(Br)cc2[nH]c(-c3ccccc3)nc12",
    "CC(C)[C@H](Nc1ncnc2[nH]cc(C(N)=O)c12)c1ccc2c(c1)OCCO2",
    "CC(C)[C@H](Nc1ncnc2[nH]c(-c3cncc(S(C)(=O)=O)c3)cc12)c1ccc2c(c1)S(=O)(=O)CCC2",
]

In [ ]:
data = api.predict(model_name=model["model_name"], smiles=data)

In [ ]:
pd.DataFrame(data)